# Render RHS

In [ ]:
#| default_exp render_rhs
%load_ext autoreload
%autoreload 2

### Overview
The searching phase of rewrite is done, so now we have a filtered ResultSet which we'll use for the transformation part.
Since we might want to change each matched graph a bit differently, depending on the actual values of its attributes, we allow the user to define a parameterized RHS string:

For each placeholder in the string, the user writes "{{name_of_placeholder}}" inside the string.
In this module, we wrap the string with jinja2's Template class and render the string with a dictionary, sent by the user to the rewrite function as **render_rhs** - a dictionary whose keys are names of templated arguments in RHS, and values are functions that receive a Match and return a new value (dependent on the current matches). For every match, we calculate the actual value that the functions specify for each template argument, and render the RHS accordingly, such that every match has its own specific RHS string.

### Requirements

In [ ]:
#| export
from typing import *
from jinja2 import Template
from graph_rewrite.match_class import Match

In [ ]:
#| export
RenderFunc = Callable[[Match], str] # type of a function to render a parameter

### Render RHS strings
Receives a filtered ResultSet, a RHS string (potentially with placeholders) and a dictionary from strings to RenderFuncs. Returns a list of RHS rendered strings, whose indices correspond to those of the ResultSet. 

In [ ]:
#| export
def render_rhs(match_list: List[Match], rhs: str, render_funcs: Dict[str, RenderFunc]) -> List[str]:
    template_rhs = Template(rhs)
    rendered_rhs: List[str] = []
    for match in match_list:
        cur_calc_args = {attr: render_funcs[attr](match) for attr in render_funcs.keys()}
        cur_rendered = template_rhs.render(cur_calc_args)
        rendered_rhs.append(cur_rendered)
    
    return rendered_rhs

### Tests

#### Test Utils

In [ ]:
# from networkx import DiGraph
# from graph_rewrite.matcher import find_matches
# from graph_rewrite.result_set import mappings_to_results_set

# def create_graph(nodes, edges):
#     g = DiGraph()
#     g.add_nodes_from(nodes)
#     g.add_edges_from(edges)
#     return g

# def get_result_set(input, pattern):
#     return mappings_to_results_set(input, pattern, find_matches(input, pattern))

#### Test Cases

In [ ]:
# input = create_graph(
#     [('A', {'attr': 5}),'B',('C', {'attr': 10}),'D'],
#     [
#         ('A','B'),
#         ('A','C'),
#         ('A', 'A'),
#         ('C', 'C'),
#         ('A', 'C')
#     ]
# )
# pattern = create_graph(['X'], [('X', 'X')])
# rs = get_result_set(input, pattern)

# rhs = "my increased attr is {{val}}"
# render_dict = {
#     "val": lambda match: match['X']['attr'] + 1
# }

# assert render_rhs(rs, rhs, render_dict) == ['my increased attr is 6', 'my increased attr is 11']